In [ ]:
#MBP의사록 discussion & content 항목 word cloud LDA
import pandas as pd
import gensim
from gensim import corpora
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import os
import re
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv(r"C:\Users\rapha\PJ\PJ_03\MPB_NNG.csv")
# content_filtered'와 'discussion_filtered' columns을 합치기
df['combined_text'] = df['content_filtered'] + " " + df['discussion_filtered']

# 한글만 추출
def filter_korean(text):
    return ' '.join(re.findall(r'[가-힣]+', text))

df['filtered_text'] = df['combined_text'].apply(filter_korean)

# 빈 줄이 있으면 삭제
df = df[df['filtered_text'].str.strip() != '']

# DTM 작성
vectorizer = CountVectorizer(tokenizer=lambda x: x.split())
dtm = vectorizer.fit_transform(df['filtered_text'])
# DTM 작성 후 DataFrame으로 변환
dtm_df = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names_out())
dtm_df.to_csv('LDA_Results/document_term_matrix.csv', index=False)

# corpus for LDA
texts = df['filtered_text'].apply(lambda x: x.split()).tolist()
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# LDA
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=30, id2word=dictionary, passes=15)

os.makedirs('LDA_Results', exist_ok=True)

# 주제별 Word Clouds
for i in range(30):  # Assuming 30 topics
    plt.figure()
    wc = WordCloud(width=1000, height=600, background_color='white', random_state=0, font_path='gulim')
    wordcloud_image = wc.generate_from_frequencies(dict(lda_model.show_topic(i, 30)))

    plt.imshow(wordcloud_image, interpolation="bilinear")
    plt.axis('off')
    plt.title(f"Topic {i+1}")
    # Save the word cloud as an image file
    plt.savefig(f'LDA_Results/topic_{i+1}_wordcloud.png')
    plt.close()

# 토픽 별 가중치 계산
for doc in corpus:
    topic_weights.append([weight for _, weight in lda_model.get_document_topics(doc)])
avg_weights = pd.DataFrame(topic_weights).mean(axis=0)
avg_weights.to_csv('LDA_Results/average_topic_weights.csv', index=False)

for i, weight in enumerate(avg_weights):
    print(f"Topic {i+1}: {weight:.4f}")
    print("Words:", [word for word, _ in lda_model.show_topic(i, 30)])
    print()

print("Word clouds saved as images and average topic weights saved as a CSV file.")